In [1]:
import torch
from torch import nn
from torch._C._te import Tensor
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 32
PCB_DATASET_LOCATION = '/work/pcb-defect-detection-histogram-approach/binary_classification_dataset'
NEW_MAX = 225
S = 16

In [3]:
def to_histogram(x: Tensor):
    x = x * 255
    yuv = torch.matmul(x, torch.tensor([
        [0.29900],
        [0.58700],
        [0.114001]
    ]))

    max_v = torch.max(yuv)
    normalized = yuv / max_v
    scaled = torch.flatten(normalized * (NEW_MAX - 1)).long()
    freq = torch.bincount(scaled, minlength=NEW_MAX)
    cum_histo = torch.cumsum(freq, dim=0)
    # normalized_freq = cum_histo / len(scaled)
    normalized_freq = freq / len(scaled)
    return normalized_freq * S

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.permute(1, 2, 0)),
    transforms.Lambda(to_histogram)
])

In [5]:
dataset = datasets.ImageFolder(PCB_DATASET_LOCATION, transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [0.9, 0.1])

In [6]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=NEW_MAX, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_relu_stack(x)


model = NeuralNetwork().to(device)
print(f"Model = {model}")

Model = NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=225, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [9]:
loss_fn = nn.BCELoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
epochs = 11

In [12]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    accuracy = (correct / len(y_pred)) * 100
    return accuracy

In [13]:
def plot_metrics(train_losses, train_accuracy, test_losses, test_accuracy):
    plt.figure(figsize=(10,5))
    plt.title("Training and Test Loss")
    plt.plot(list(map(lambda x : x.detach().numpy(), test_losses)), label="test")
    plt.plot(list(map(lambda x : x.detach().numpy(), train_losses)), label="train")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10,5))
    plt.title("Training and Test Accuracy")
    plt.plot(test_accuracy, label="test")
    plt.plot(train_accuracy, label="train")
    plt.xlabel("iterations")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

In [14]:
train_losses = []
train_accuracy = []
test_losses = []
test_accuracy = []

In [15]:
for epoch in range(epochs):
    train_accuracy_sum = 0
    count_batches = 0
    sum_loss_train = 0

    for _, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        image, labels = data

        y_logits = model(image).squeeze(1)
        y_pred = torch.round(y_logits)

        # 2. Calculate loss/accuracy
        loss = loss_fn(y_logits, labels.float())
        sum_loss_train += loss
        train_accuracy_sum += accuracy_fn(y_true=labels, y_pred=y_pred)
        count_batches += 1

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backwards
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    acc = train_accuracy_sum / count_batches
    train_loss = sum_loss_train / count_batches

    test_accuracy_sum = 0
    test_count_batches = 0
    sum_loss_test = 0

    model.eval()
    with torch.inference_mode():
        for _, data in tqdm(enumerate(val_loader), total=len(val_loader)):
            image, labels = data
            test_logits = model(image).squeeze(1)
            test_pred = torch.round(torch.sigmoid(test_logits))
            test_loss = loss_fn(test_logits, labels.float())
            sum_loss_test += test_loss
            test_count_batches += 1
            test_accuracy_sum += accuracy_fn(y_true=labels, y_pred=test_pred)

    test_loss_avg = sum_loss_test / test_count_batches
    test_acc = test_accuracy_sum / test_count_batches

    train_losses.append(train_loss)
    train_accuracy.append(acc)
    test_losses.append(test_loss_avg)
    test_accuracy.append(test_acc)

    print(f"Epoch: {epoch} | Loss: {train_loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss_avg:.5f}, Test acc: {test_acc:.2f}%")


  0%|          | 0/85 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plot_metrics(train_losses, train_accuracy, test_losses, test_accuracy)